In [1]:
import pandas as pd
import numpy as np
import os
import Memory_Collaborative_Filtering as mem
import sqlite3 as sql
import sklearn 
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import pickle

In [2]:
path = 'C:/Users/arjun/Desktop/amazon.db'
path2 = 'C:/Users/arjun/Desktop/frequency_rankings.db'

In [30]:
review_data = mem.import_standard_data(path)
review_data = review_data.drop_duplicates(['customer_id', 'product_id'])
review_data['star_rating'] = review_data['star_rating'].astype(int)
review_data['review_length'] = review_data['review_body'].str.count(' ') + 1
review_data = pd.merge(review_data, user_counts_top_10000, left_on='customer_id', right_on='customer_id', how='inner')

In [34]:
print(review_data.columns)

Index(['customer_id', 'product_id', 'product_title', 'review_body',
       'star_rating', 'review_length', 'counts'],
      dtype='object')


In [35]:
final = review_data[['star_rating', 'review_length', 'counts']]

In [36]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm = 'ball_tree').fit(final)

In [37]:
distances2, indices2 = nbrs.kneighbors(final)

In [65]:
print(distances2)

[[0.         1.         1.41421356 2.         2.23606798 2.44948974]
 [0.         2.         2.23606798 2.44948974 3.         3.        ]
 [0.         1.41421356 1.41421356 1.73205081 2.         2.23606798]
 ...
 [0.         1.41421356 1.41421356 1.73205081 2.         2.        ]
 [0.         0.         1.         1.         1.         1.        ]
 [0.         0.         0.         0.         1.41421356 1.41421356]]


In [66]:
print(indices2)

[[    0 43729 77521  8670 86416 21334]
 [    1    24  8679 29255 66676 75077]
 [    2    85 77538 22943    69 15690]
 ...
 [95767 70782 46372 37504 90168 78011]
 [95768 95437 20262 23057 43912 42786]
 [20245 95769 80713 43909  9274 65139]]


In [55]:
def get_index_from_name(name):
    return review_data[review_data["product_title"]==name].index.tolist()[0]

In [56]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [57]:
def print_similar_books(query=None,id=None, model):
    distances2, indices2 = model.kneighbors()
    if id:
        for id in indices2[id][1:]:
            print(review_data.ix[id]["product_title"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices2[found_id][1:]:
            print(review_data.ix[id]["product_title"])

In [63]:
print_similar_books(query="The Choice")

The Curious Incident of the Dog in the Night-Time
Waiting On You (Blue Heron)
Istanbul Passage: A Novel
The Innocent: A Novel
The Witch with No Name


C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [64]:
filename = 'finalized_model.sav'
pickle.dump(nbrs, open(filename, 'wb'))

In [62]:
print(review_data['product_title'].head(10000))

0                                 Trial Run (Fault Lines)
1                  Through Waters Deep (Waves of Freedom)
2       Secret Garden: An Inky Treasure Hunt and Color...
3                                         A Bride at Last
4       Among the Fair Magnolias: Four Southern Love S...
5                                           Summer's List
6                                   The Innocent: A Novel
7       Once Upon a Summertime: A New York City Romanc...
8            Two Roads Home: A Chicory Inn Novel - Book 2
9                     Hearts Made Whole (Beacons of Hope)
10                                           Fatal Trauma
11      The Proposal at Siesta Key: Amish Brides of Pi...
12      Every Bride Needs a Groom: A Novel (Brides wit...
13      A Sparrow in Terezin (A Hidden Masterpiece Novel)
14           How to Catch a Prince (Royal Wedding Series)
15      NKJV, Duck Commander Faith and Family Bible, H...
16      Shadows of the Past: A Novel (Logan Point) (Vo...
17      Dare t

In [ ]:
review_data_pivot = review_data.pivot(index = 'product_id', columns = 'customer_id', values = 'star_rating').fillna(0)
review_data_matrix = csr_matrix(review_data_pivot.values)

In [19]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(review_data_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [20]:
query_index = review_data_pivot[1:].shape[0]

In [21]:
distances, indices = model_knn.kneighbors(review_data_pivot.iloc[[query_index]], n_neighbors = 5)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(review_data_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, review_data_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for B00MPREWL4:

1: 0515154113, with distance of 0.5648058601107555:
2: 1416549595, with distance of 0.5917517095361371:
3: 1455577847, with distance of 0.5917517095361371:
4: 0385739176, with distance of 0.7113248654051871:


In [22]:
test = review_data['product_id']
test = test.drop_duplicates()
test = test.reset_index(drop=True)
print(len(test))

9254
